In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
! pip install accelerate peft transformers datasets seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00:00:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 24.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 12.4 MB/s eta 0:00:00:00:010:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.8 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 67.6 MB/s eta 0:00:00:00:0100:01
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=6d32c1fcd12d86bc3038f83e233d54ce3a214bd7ac304ee46a06b89f688500a5
  Stored 

In [2]:
# Load the dataset (This dataset contains NER Tags belonging to Person, Location, Org, Misc)
from datasets import load_dataset
dataset = load_dataset("conll2003")

README.md:   0%|          | 0.00/12.3k [00:00<?, ?B/s]

conll2003.py:   0%|          | 0.00/9.57k [00:00<?, ?B/s]

The repository for conll2003 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/conll2003.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N]  y


Generating train split:   0%|          | 0/14041 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3250 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3453 [00:00<?, ? examples/s]

In [18]:
# Sample Data from Train section
dataset["train"][0]

{'id': '0',
 'tokens': ['EU',
  'rejects',
  'German',
  'call',
  'to',
  'boycott',
  'British',
  'lamb',
  '.'],
 'pos_tags': [22, 42, 16, 21, 35, 37, 16, 21, 7],
 'chunk_tags': [11, 21, 11, 12, 21, 22, 11, 12, 0],
 'ner_tags': [3, 0, 7, 0, 0, 0, 7, 0, 0]}

In [15]:
# Extract the feature names related to NER Tags

label_list = dataset["train"].features["ner_tags"].feature.names
num_labels = len(label_list)
print(label_list)
print(num_labels)

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']
9


### Finetuning Base BERT Model for NER Task

In [16]:
from transformers import BertTokenizerFast

# BERT Tokenizer fetch with Base BERT Model
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
i like running                  # Input Text

cls i like run ##ing sep        # Tokens by BERT Tokenizer

None 0 1 2 2 None               # Word Indices

-100 7 7 3 -100 -100            # Ground Labels for Training


In [23]:
# Preprocessing Definition to get the above cell functionality
# is_split_into_words = True means the words are being split into sub tokens by BERT
# Append a huge -ve value for consecutively repeated, CLS, SEP Tokens to not make them participate
# For others append the respective NER Tag

def tokenize(example):
    tokenized_inputs = tokenizer(example["tokens"], truncation=True, is_split_into_words=True)
    labels = []
    prev_word_idx = None
    word_ids = tokenized_inputs.word_ids()
    for word_idx in word_ids:
        if word_idx is None:
            labels.append(-100)
        elif word_idx != prev_word_idx:
            labels.append(example["ner_tags"][word_idx])
        else:
            labels.append(-100)
        prev_word_idx = word_idx
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

# Map the function to the dataset
tokenized_dataset = dataset.map(tokenize, batched=)

Map:   0%|          | 0/14041 [00:00<?, ? examples/s]

Map:   0%|          | 0/3250 [00:00<?, ? examples/s]

Map:   0%|          | 0/3453 [00:00<?, ? examples/s]

In [25]:
from transformers import BertForTokenClassification

# BertForTokenClassification module to get the final layer logits
model = BertForTokenClassification.from_pretrained("bert-base-uncased", num_labels=num_labels)


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [29]:
# LoRA Finetuning at Query and Value attention layers

from peft import LoraConfig, TaskType, get_peft_model

lora_config = LoraConfig(
    r = 8,                              # LoRA Rank (d*r, r*d)
    lora_dropout = 0.1,                 # LoRA Dropout
    lora_alpha = 32,                    # LoRA Alpha
    target_modules = ["query","value"], # Target Modules
    bias = "none",                      # Bias is set to none
    task_type = TaskType.TOKEN_CLS,     # Token Classification
)

model = get_peft_model(model, lora_config)    # Apply the LoRA Configurations to the base model
model.print_trainable_parameters()

trainable params: 301,833 || all params: 109,200,402 || trainable%: 0.2764


In [32]:
!pip install evaluate

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.3 MB/s eta 0:00:00


In [33]:
from transformers import TrainingArguments, Trainer, DataCollatorForTokenClassification
import evaluate

# Load the seqeval metric
seqeval = evaluate.load("seqeval")

def compute_metrics(p):
    predictions, labels = p

    # Step 1: Take the argmax over the last dimension (convert logits to predicted class indices)
    predictions = predictions.argmax(-1)

    # Step 2: Initialize containers for the cleaned, label-name-based predictions and labels
    true_predictions = []
    true_labels = []

    # Step 3: Iterate over each example in the batch
    for prediction, label in zip(predictions, labels):
        sentence_preds = []
        sentence_labels = []

        # Step 4: Iterate over each token in the sentence
        for p_token, l_token in zip(prediction, label):
            if l_token != -100:  # Ignore padding or special tokens
                sentence_preds.append(label_list[p_token])  # Convert predicted index to label name
                sentence_labels.append(label_list[l_token])  # Convert true label index to label name

        # Step 5: Add the sentence's prediction and label lists to the batch lists
        true_predictions.append(sentence_preds)
        true_labels.append(sentence_labels)

    # Step 6: Compute metrics using seqeval
    return seqeval.compute(predictions=true_predictions, references=true_labels)


In [35]:
# Training  Arguments required for Trainer Class

training_args = TrainingArguments(
    output_dir="./bert-ner-lora",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-4,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
)

# Data Collator use to make batches
data_collator = DataCollatorForTokenClassification(tokenizer)

# Trainer class to make the model train
trainer = Trainer(
    model=model,                              # LoRA Finetuned Model
    args=training_args,                       # Training Arguments
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,                      # BERT Tokenizer
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

/tmp/ipykernel_35/570852030.py:16: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForTokenClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [40]:
# WANDB Login for training using API

import wandb

wandb.login(key="***************************************")
wandb.init(project="bert-ner-lora", name="run-1")

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


In [41]:
# Training starts here

trainer.train()

Epoch,Training Loss,Validation Loss,Loc,Misc,Org,Per,Overall Precision,Overall Recall,Overall F1,Overall Accuracy
1,0.060100,0.069751,"{'precision': 0.8868302453680521, 'recall': 0.9640718562874252, 'f1': 0.9238393322900365, 'number': 1837}","{'precision': 0.7561837455830389, 'recall': 0.6963123644251626, 'f1': 0.7250141163184641, 'number': 922}","{'precision': 0.8081081081081081, 'recall': 0.8918717375093214, 'f1': 0.847926267281106, 'number': 1341}","{'precision': 0.9721006564551422, 'recall': 0.9647122692725298, 'f1': 0.9683923705722071, 'number': 1842}",0.875203,0.906429,0.890542,0.979615
2,0.045200,0.054185,"{'precision': 0.9369994660971703, 'recall': 0.9553620032661949, 'f1': 0.9460916442048517, 'number': 1837}","{'precision': 0.7984749455337691, 'recall': 0.7950108459869848, 'f1': 0.7967391304347826, 'number': 922}","{'precision': 0.8893838158871566, 'recall': 0.8933631618195377, 'f1': 0.8913690476190477, 'number': 1341}","{'precision': 0.969551282051282, 'recall': 0.9853420195439739, 'f1': 0.9773828756058157, 'number': 1842}",0.915308,0.925783,0.920515,0.984911
3,0.059000,0.050439,"{'precision': 0.9432851792402355, 'recall': 0.95971692977681, 'f1': 0.9514301133297356, 'number': 1837}","{'precision': 0.8004291845493562, 'recall': 0.8091106290672451, 'f1': 0.8047464940668824, 'number': 922}","{'precision': 0.8877551020408163, 'recall': 0.9082774049217002, 'f1': 0.8978990047917434, 'number': 1341}","{'precision': 0.9746083198271205, 'recall': 0.9793702497285559, 'f1': 0.9769834822637422, 'number': 1842}",0.918161,0.930831,0.924453,0.985631


Trainer is attempting to log a value of "{'precision': 0.8868302453680521, 'recall': 0.9640718562874252, 'f1': 0.9238393322900365, 'number': 1837}" of type <class 'dict'> for key "eval/LOC" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.7561837455830389, 'recall': 0.6963123644251626, 'f1': 0.7250141163184641, 'number': 922}" of type <class 'dict'> for key "eval/MISC" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.8081081081081081, 'recall': 0.8918717375093214, 'f1': 0.847926267281106, 'number': 1341}" of type <class 'dict'> for key "eval/ORG" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.9721006564551422, 'recall': 0.9647122692725298,

TrainOutput(global_step=1317, training_loss=0.09569618590147819, metrics={'train_runtime': 298.0475, 'train_samples_per_second': 141.33, 'train_steps_per_second': 4.419, 'total_flos': 1133709322521564.0, 'train_loss': 0.09569618590147819, 'epoch': 3.0})

### Inference using Finetuned BERT for NER Task

In [47]:
# Save Model and Tokenizer

model.save_pretrained("./bert-ner-lora-adapters")
tokenizer.save_pretrained("./bert-ner-lora-adapters")

('./bert-ner-lora-adapters/tokenizer_config.json',
 './bert-ner-lora-adapters/special_tokens_map.json',
 './bert-ner-lora-adapters/vocab.txt',
 './bert-ner-lora-adapters/added_tokens.json',
 './bert-ner-lora-adapters/tokenizer.json')

In [48]:
# Applying Tokenizer and BertForTokenClassification on the fine - tuned model

from transformers import BertForTokenClassification, BertTokenizerFast

model_path = "/kaggle/working/bert-ner-lora-adapters"
num_labels = 9     # Number of NER Classes in Dataset = 9

model = BertForTokenClassification.from_pretrained(model_path, num_labels=num_labels)
tokenizer = BertTokenizerFast.from_pretrained(model_path)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [69]:
# Moving the model to GPU if available
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): lora.Linear(
                (base_layer): Linear(in_features=768, out_features=768, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=768, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=

In [67]:
# Test Sentence
sentence = "EU rejects German call to boycott British lamb."
tokens = tokenizer.tokenize(sentence)
print(tokens)


['eu', 'rejects', 'german', 'call', 'to', 'boycott', 'british', 'lamb', '.']


In [70]:
# Inference - Forward Pass on the model

inputs = tokenizer(sentence, return_tensors="pt")
inputs = {k: v.to(device) for k, v in inputs.items()}
outputs = model(**inputs)
logits = outputs.logits
print(logits)

tensor([[[ 6.5184, -2.1567, -5.0326, -0.2609, -3.3490, -0.6522, -4.5866,
          -1.8128, -4.2013],
         [-0.0180, -1.0776, -3.2866,  5.2628, -1.7039,  3.9776, -2.3396,
           0.6239, -3.9333],
         [ 7.4077, -4.3776, -4.8220, -2.2940, -1.8127, -3.9632, -3.9071,
          -2.5397, -2.3702],
         [ 0.5706, -1.6308, -3.9131,  0.7335, -3.2838,  0.7335, -2.9677,
           6.8644, -0.0675],
         [ 7.5067, -4.0407, -5.1054, -2.2085, -2.0101, -3.8528, -4.1749,
          -1.7184, -2.4478],
         [ 7.8290, -4.0385, -5.0807, -2.3046, -2.2652, -3.8299, -4.1231,
          -2.4784, -3.0555],
         [ 7.6783, -3.2521, -4.8373, -1.6185, -2.5400, -3.2518, -4.4885,
          -1.8913, -3.3188],
         [ 0.4136, -1.6337, -3.3009,  0.5506, -3.0407,  1.2399, -2.8815,
           6.6247, -0.5326],
         [ 6.2841, -3.4898, -4.4371, -2.4205, -0.6604, -3.1009, -2.8188,
          -2.5737, -2.2779],
         [ 7.9261, -3.2961, -5.0573, -1.5554, -2.8602, -2.2882, -4.6617,
         

In [71]:
# Taking the max logit for each token output

predictions = torch.argmax(logits, dim = -1)
predicted_label_ids = predictions[0].tolist()
label_list = model.config.id2label
predicted_labels = [label_list[i] for i in predicted_label_ids]

In [75]:
# For reference

label_list = dataset["train"].features["ner_tags"].feature.names
dict_temp = {}
flag = 0
for x in label_list:
    dict_temp[x] = flag
    flag += 1
dict_temp

{'O': 0,
 'B-PER': 1,
 'I-PER': 2,
 'B-ORG': 3,
 'I-ORG': 4,
 'B-LOC': 5,
 'I-LOC': 6,
 'B-MISC': 7,
 'I-MISC': 8}

In [77]:
# Final result and inferenced output

tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])
for token, label in zip(tokens, predicted_labels):
    print(f"{token:15} -> {label}")

[CLS]           -> LABEL_0
eu              -> LABEL_3
rejects         -> LABEL_0
german          -> LABEL_7
call            -> LABEL_0
to              -> LABEL_0
boycott         -> LABEL_0
british         -> LABEL_7
lamb            -> LABEL_0
.               -> LABEL_0
[SEP]           -> LABEL_0
